This notebook is used to predict electricity prices (HOEP in Ontario) with XGBoost model. Different features are created from price, demand and temperature data. For each year, 4 XGBoost models are created to get price forecast for 1,6,12 and 24 hours in the future. To do so, we restrict the number of features used as inputs in the model. The model is trained on 2 weeks of data and is then tested on the next 48 hours which are used as forecasts for the price.

In [1]:
#### !pip3 install xgboost

import pandas as pd 
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from scipy.stats import uniform, loguniform, randint

# Prepare data
df = pd.read_csv("HOEP_2015-2019_dataset.csv")
df['Datetime'] = pd.to_datetime(df.Date) + pd.to_timedelta(df.Hour, unit='h')

# create new features
df['HOEP_d12-24'] = df.HOEP.shift(12) - df.HOEP.shift(24)
df['HOEP_d12-24_lag1'] = df['HOEP_d12-24'].shift(1)
df['HOEP_d12-24_lag2'] = df['HOEP_d12-24'].shift(2)
df['HOEP_d12-24_lag3'] = df['HOEP_d12-24'].shift(3)
df['HOEP_d12-24_lag4'] = df['HOEP_d12-24'].shift(4)
df['HOEP_d12-24_lag5'] = df['HOEP_d12-24'].shift(5)
df['HOEP_d12-24_lag6'] = df['HOEP_d12-24'].shift(6)
df['HOEP_d12-24_lag7'] = df['HOEP_d12-24'].shift(7)
df['HOEP_d12-24_lag8'] = df['HOEP_d12-24'].shift(8)
df['HOEP_d12-24_lag9'] = df['HOEP_d12-24'].shift(9)
df['HOEP_d12-24_lag10'] = df['HOEP_d12-24'].shift(10)
df['HOEP_d12-24_lag11'] = df['HOEP_d12-24'].shift(11)
df['HOEP_d12-24_lag12'] = df['HOEP_d12-24'].shift(12)

df['MktDem_d12-24'] = df.MktDem.shift(12) - df.MktDem.shift(24)
df['MktDem_d12-24_lag1'] = df['MktDem_d12-24'].shift(1)
df['MktDem_d12-24_lag2'] = df['MktDem_d12-24'].shift(2)
df['MktDem_d12-24_lag3'] = df['MktDem_d12-24'].shift(3)
df['MktDem_d12-24_lag4'] = df['MktDem_d12-24'].shift(4)
df['MktDem_d12-24_lag5'] = df['MktDem_d12-24'].shift(5)
df['MktDem_d12-24_lag6'] = df['MktDem_d12-24'].shift(6)
df['MktDem_d12-24_lag7'] = df['MktDem_d12-24'].shift(7)
df['MktDem_d12-24_lag8'] = df['MktDem_d12-24'].shift(8)
df['MktDem_d12-24_lag9'] = df['MktDem_d12-24'].shift(9)
df['MktDem_d12-24_lag10'] = df['MktDem_d12-24'].shift(10)
df['MktDem_d12-24_lag11'] = df['MktDem_d12-24'].shift(11)
df['MktDem_d12-24_lag12'] = df['MktDem_d12-24'].shift(12)

# create news features
df['Temp(C)_d12-24'] = df['Temp(C)'].shift(12) - df['Temp(C)'].shift(24)
df['Temp(C)_d12-24_lag1'] = df['Temp(C)_d12-24'].shift(1)
df['Temp(C)_d12-24_lag2'] = df['Temp(C)_d12-24'].shift(2)
df['Temp(C)_d12-24_lag3'] = df['Temp(C)_d12-24'].shift(3)
df['Temp(C)_d12-24_lag4'] = df['Temp(C)_d12-24'].shift(4)
df['Temp(C)_d12-24_lag5'] = df['Temp(C)_d12-24'].shift(5)
df['Temp(C)_d12-24_lag6'] = df['Temp(C)_d12-24'].shift(6)
df['Temp(C)_d12-24_lag7'] = df['Temp(C)_d12-24'].shift(7)
df['Temp(C)_d12-24_lag8'] = df['Temp(C)_d12-24'].shift(8)
df['Temp(C)_d12-24_lag9'] = df['Temp(C)_d12-24'].shift(9)
df['Temp(C)_d12-24_lag10'] = df['Temp(C)_d12-24'].shift(10)
df['Temp(C)_d12-24_lag11'] = df['Temp(C)_d12-24'].shift(11)
df['Temp(C)_d12-24_lag12'] = df['Temp(C)_d12-24'].shift(12)

# create news features
df['HOEP_d24-48'] = df.HOEP.shift(24) - df.HOEP.shift(48)
df['HOEP_d24-48_lag1'] = df['HOEP_d24-48'].shift(1)
df['HOEP_d24-48_lag2'] = df['HOEP_d24-48'].shift(2)
df['HOEP_d24-48_lag3'] = df['HOEP_d24-48'].shift(3)
df['HOEP_d24-48_lag4'] = df['HOEP_d24-48'].shift(4)
df['HOEP_d24-48_lag5'] = df['HOEP_d24-48'].shift(5)
df['HOEP_d24-48_lag6'] = df['HOEP_d24-48'].shift(6)
df['HOEP_d24-48_lag7'] = df['HOEP_d24-48'].shift(7)
df['HOEP_d24-48_lag8'] = df['HOEP_d24-48'].shift(8)
df['HOEP_d24-48_lag9'] = df['HOEP_d24-48'].shift(9)
df['HOEP_d24-48_lag10'] = df['HOEP_d24-48'].shift(10)
df['HOEP_d24-48_lag11'] = df['HOEP_d24-48'].shift(11)
df['HOEP_d24-48_lag12'] = df['HOEP_d24-48'].shift(12)

df['MktDem_d24-48'] = df.MktDem.shift(24) - df.MktDem.shift(48)
df['MktDem_d24-48_lag1'] = df['MktDem_d24-48'].shift(1)
df['MktDem_d24-48_lag2'] = df['MktDem_d24-48'].shift(2)
df['MktDem_d24-48_lag3'] = df['MktDem_d24-48'].shift(3)
df['MktDem_d24-48_lag4'] = df['MktDem_d24-48'].shift(4)
df['MktDem_d24-48_lag5'] = df['MktDem_d24-48'].shift(5)
df['MktDem_d24-48_lag6'] = df['MktDem_d24-48'].shift(6)
df['MktDem_d24-48_lag7'] = df['MktDem_d24-48'].shift(7)
df['MktDem_d24-48_lag8'] = df['MktDem_d24-48'].shift(8)
df['MktDem_d24-48_lag9'] = df['MktDem_d24-48'].shift(9)
df['MktDem_d24-48_lag10'] = df['MktDem_d24-48'].shift(10)
df['MktDem_d24-48_lag11'] = df['MktDem_d24-48'].shift(11)
df['MktDem_d24-48_lag12'] = df['MktDem_d24-48'].shift(12)

df['MktDem_lag360'] = df.MktDem.shift(360)
df['MktDem_lag168'] = df.MktDem.shift(168)

df.columns

Index(['Date', 'Hour', 'Year', 'Month', 'Weekday', 'HOEP', 'MktDem', 'OntDem',
       'Temp(C)', 'Stn_Press(kPa)',
       ...
       'MktDem_d24-48_lag5', 'MktDem_d24-48_lag6', 'MktDem_d24-48_lag7',
       'MktDem_d24-48_lag8', 'MktDem_d24-48_lag9', 'MktDem_d24-48_lag10',
       'MktDem_d24-48_lag11', 'MktDem_d24-48_lag12', 'MktDem_lag360',
       'MktDem_lag168'],
      dtype='object', length=225)

In [2]:
# Data that will be kept as features for the model

forecast = [] # create a list to store predicted values for HOEP
forecast_time = []
features_list = []
importance_list = []
RMSE_list = []
f_imp_list = []

# define X and y 
columns = list(df)

# items to be removed if we want forecasts for the next period only
unwanted_X_1 = {
'Datetime', 'HOEP', 'Temp(C)', 'Date', 'Stn_Press(kPa)', 'MktDem', 'OntDem',
             } 
# items to be removed if we want forecasts for the next 6 periods
unwanted_X_6 = {
'Datetime', 'HOEP', 'Temp(C)', 'Date', 'Stn_Press(kPa)', 'MktDem', 'OntDem',
    
'HOEP_lag1','HOEP_lag2','HOEP_lag3','HOEP_lag4', 'HOEP_lag5',
'MktDem_lag1','MktDem_lag2','MktDem_lag3','MktDem_lag4','MktDem_lag5',
'Temp(C)_lag1','Temp(C)_lag2','Temp(C)_lag3','Temp(C)_lag4','Temp(C)_lag5',
}

# items to be removed if we want forecasts for the next 12 periods
unwanted_X_12 = {
'Datetime', 'HOEP', 'Temp(C)', 'Date', 'Stn_Press(kPa)', 'MktDem', 'OntDem',
    
'HOEP_lag1','HOEP_lag2','HOEP_lag3','HOEP_lag4', 'HOEP_lag5','HOEP_lag6','HOEP_lag7',
'HOEP_lag8','HOEP_lag9','HOEP_lag10','HOEP_lag11',

'MktDem_lag1','MktDem_lag2','MktDem_lag3','MktDem_lag4','MktDem_lag5','MktDem_lag6','MktDem_lag7',
'MktDem_lag8','MktDem_lag9','MktDem_lag10','MktDem_lag11',
              
'Temp(C)_lag1','Temp(C)_lag2','Temp(C)_lag3','Temp(C)_lag4','Temp(C)_lag5','Temp(C)_lag6',
'Temp(C)_lag7','Temp(C)_lag8','Temp(C)_lag9','Temp(C)_lag10','Temp(C)_lag11',
             } 

# items to be removed if we want forecasts for the next 24 periods
unwanted_X_24 = {
'Datetime', 'HOEP', 'Temp(C)', 'Date', 'Stn_Press(kPa)', 'MktDem', 'OntDem',
    
'HOEP_lag1','HOEP_lag2','HOEP_lag3','HOEP_lag4', 'HOEP_lag5','HOEP_lag6','HOEP_lag7',
'HOEP_lag8','HOEP_lag9','HOEP_lag10','HOEP_lag11','HOEP_lag12',
'HOEP_lag13','HOEP_lag14','HOEP_lag15','HOEP_lag16','HOEP_lag17','HOEP_lag18','HOEP_lag19',
'HOEP_lag20','HOEP_lag21', 'HOEP_lag22', 'HOEP_lag23',

'MktDem_lag1','MktDem_lag2','MktDem_lag3','MktDem_lag4','MktDem_lag5','MktDem_lag6','MktDem_lag7',
'MktDem_lag8','MktDem_lag9','MktDem_lag10','MktDem_lag11','MktDem_lag12',
'MktDem_lag13','MktDem_lag14','MktDem_lag15','MktDem_lag16','MktDem_lag17','MktDem_lag18'
'MktDem_lag19','MktDem_lag20','MktDem_lag21','MktDem_lag22','MktDem_lag23', 
              
'Temp(C)_lag1','Temp(C)_lag2','Temp(C)_lag3','Temp(C)_lag4','Temp(C)_lag5','Temp(C)_lag6',
'Temp(C)_lag7','Temp(C)_lag8','Temp(C)_lag9','Temp(C)_lag10','Temp(C)_lag11','Temp(C)_lag12',
'Temp(C)_lag13','Temp(C)_lag14','Temp(C)_lag15','Temp(C)_lag16','Temp(C)_lag17','Temp(C)_lag18',
'Temp(C)_lag19','Temp(C)_lag20','Temp(C)_lag21','Temp(C)_lag22','Temp(C)_lag23', 
              
'HOEP_d12-24','MktDem_d12-24', 'Temp(C)_d12-24', 
'HOEP_d12-24_lag1','MktDem_d12-24_lag1', 'Temp(C)_d12-24_lag1',
'HOEP_d12-24_lag2','MktDem_d12-24_lag2', 'Temp(C)_d12-24_lag2',
'HOEP_d12-24_lag3','MktDem_d12-24_lag3', 'Temp(C)_d12-24_lag3',
'HOEP_d12-24_lag4','MktDem_d12-24_lag4', 'Temp(C)_d12-24_lag4',
'HOEP_d12-24_lag5','MktDem_d12-24_lag5', 'Temp(C)_d12-24_lag5',
'HOEP_d12-24_lag6','MktDem_d12-24_lag6', 'Temp(C)_d12-24_lag6',
'HOEP_d12-24_lag7','MktDem_d12-24_lag7', 'Temp(C)_d12-24_lag7',
'HOEP_d12-24_lag8','MktDem_d12-24_lag8', 'Temp(C)_d12-24_lag8',
'HOEP_d12-24_lag9','MktDem_d12-24_lag9', 'Temp(C)_d12-24_lag9',
'HOEP_d12-24_lag10','MktDem_d12-24_lag10', 'Temp(C)_d12-24_lag10',
'HOEP_d12-24_lag11','MktDem_d12-24_lag11', 'Temp(C)_d12-24_lag11',
'HOEP_d12-24_lag12','MktDem_d12-24_lag12', 'Temp(C)_d12-24_lag12',
             } 

# Choose unwanted_X_1, unwanted_X_6, unwanted_X_12 or unwanted_X_24 depending on type of forecast we want.
columns = [ele for ele in columns if ele not in unwanted_X_24] 
columns

['Hour',
 'Year',
 'Month',
 'Weekday',
 'Season',
 'HOEP_lag24',
 'HOEP_lag25',
 'HOEP_lag26',
 'HOEP_lag27',
 'HOEP_lag28',
 'HOEP_lag29',
 'HOEP_lag30',
 'HOEP_lag31',
 'HOEP_lag32',
 'HOEP_lag33',
 'HOEP_lag34',
 'HOEP_lag35',
 'HOEP_lag36',
 'HOEP_lag37',
 'HOEP_lag38',
 'HOEP_lag39',
 'HOEP_lag40',
 'HOEP_lag41',
 'HOEP_lag42',
 'HOEP_lag43',
 'HOEP_lag44',
 'HOEP_lag45',
 'HOEP_lag46',
 'HOEP_lag47',
 'HOEP_lag48',
 'MktDem_lag18',
 'MktDem_lag19',
 'MktDem_lag24',
 'MktDem_lag25',
 'MktDem_lag26',
 'MktDem_lag27',
 'MktDem_lag28',
 'MktDem_lag29',
 'MktDem_lag30',
 'MktDem_lag31',
 'MktDem_lag32',
 'MktDem_lag33',
 'MktDem_lag34',
 'MktDem_lag35',
 'MktDem_lag36',
 'MktDem_lag37',
 'MktDem_lag38',
 'MktDem_lag39',
 'MktDem_lag40',
 'MktDem_lag41',
 'MktDem_lag42',
 'MktDem_lag43',
 'MktDem_lag44',
 'MktDem_lag45',
 'MktDem_lag46',
 'MktDem_lag47',
 'MktDem_lag48',
 'Temp(C)_lag24',
 'Temp(C)_lag25',
 'Temp(C)_lag26',
 'Temp(C)_lag27',
 'Temp(C)_lag28',
 'Temp(C)_lag29',
 'Temp(

In [ ]:
# Fit the XGB model on specific period 
from datetime import datetime
import itertools

start_time = datetime.now()

# Choose start period as either 2019-01-01 01:00:00 or 2018 or 2017 or 2016 ... 
start = df[df.Datetime == '2016-01-01 01:00:00'].index.item()
forecast_horizon = 48

for i in range(0,183): # we se range up to 183 because (8760-48) / 48 hours = 181.5
    c = start+(forecast_horizon*i) # beginning index of test dataset
    d = (start+forecast_horizon-1)+(forecast_horizon*i) # to get test data size = 2 days or 48 hours
    b = c-1
    a = b-359 # to get training data size = 2 weeks or 360 hours. a=beginning of train dataset

    data = df.loc[a:d]
    X = data[columns] # All selected variables other than HOEP
    y = data['HOEP']

    data_dmatrix = xgb.DMatrix(data=X,label=y)

    X_train = X.loc[a:b]
    y_train = y.loc[a:b]
    X_test = X.loc[c:d]
    y_test = y.loc[c:d]
    
    xgb_model = xgb.XGBRegressor() 

    param_search = {
        'max_depth': randint(5, 20),
        'min_child_weight': uniform(1.5, 1),
        'learning_rate': loguniform(1e-4, 1e0),
        'gamma': uniform(5, 5)
    }

    tscv = TimeSeriesSplit(n_splits=10)

    rsearch = RandomizedSearchCV(
        estimator=xgb_model,
        cv=tscv,
        param_distributions=param_search,
        verbose=True,
        n_jobs=-1,    
        random_state=42  # To get the same results every time we run it (replicability)
    ).fit(X_train, y_train)

    report(rsearch.cv_results_)
    print("Best Model params are:", rsearch.best_params_)

    # ESTIMATION
    xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', gamma = rsearch.best_params_['gamma'],
                                        learning_rate = rsearch.best_params_['learning_rate'], 
                                        max_depth = rsearch.best_params_['max_depth'],
                                        min_child_weight = rsearch.best_params_['min_child_weight'])
    xg_reg.fit(X_train,y_train)
    preds = xg_reg.predict(X_test)
    RMSE_xgb = np.sqrt(mean_squared_error(y_test, preds))
    print(RMSE_xgb)
    
    # PLOTS
    plt.figure(figsize=(20,5))
    plt.plot(y_test.values, c='b', label='y_test', lw=3, alpha=0.5)
    plt.plot(preds, '--', c='r', label='y_pred', lw=2)
    plt.legend()
    plt.grid()
    plt.title('HOEP Forecast (MWh)')
    plt.xlabel('time')
    plt.ylabel('MWh')
    plt.show()

    # FEATURE IMPORTANCE
    features_importances = xg_reg.feature_importances_
    f_imp = list(features_importances)
    features = pd.DataFrame(columns, columns=['feature'])
    features['importance'] = f_imp
    largest = features.nlargest(12, ['importance']) 

    plt.figure(figsize=(20,5))
    plt.bar(largest.feature,largest.importance)
    plt.ylabel("Feature Importance")
    
    # Collect and append results for each elements of the loop
    data['Datetime'] = pd.to_datetime(data.Date) + pd.to_timedelta(data.Hour, unit='h')
    forecast_time.append(data.Datetime.loc[c:d]) # we create datetime after fitting the model because 
    forecast.append(preds)                       # XGBoost doesn't allow categorical input like datetime 
    features_list.append(largest.feature)
    importance_list.append(largest.importance)
    RMSE_list.append(RMSE_xgb)
    f_imp_list.append(f_imp)

# Transform the lists of results in dataframe we can export    
merged_forecast = list(itertools.chain(*forecast))
merged_Datetime = list(itertools.chain(*forecast_time))
results = pd.DataFrame(merged_forecast, columns=['Forecast'])
results['Datetime'] = merged_Datetime
results = results.loc[:8759] # keep only results for the whole year

print("Running time:", datetime.now() - start_time)
print("RMSE mean:",np.mean(RMSE_list))
results

In [ ]:
# Exports dataframe to csv
results.to_csv (r'C:\Users\Administrateur\Dropbox\Python\Battery algorithms\Thèse\Travail\Algorithm\RESULTS\value_storage\HOEP_2016_forecast_XGB24.csv', 
           index = False, header=True)

results

In [ ]:
# Plot relation between HOEP and temperature
%matplotlib inline  
import seaborn as sns
import matplotlib 
import matplotlib.pyplot as plt 
sns.set() # Use seaborn style
from scipy import stats

perc = stats.percentileofscore(df.HOEP, 94)
print("% of data kept for HOEP plot:", round(perc,5))

with sns.axes_style("ticks", {"grid.color": "white"}):
    fig, ax = plt.subplots()
    g = sns.scatterplot(data=df,x='Temp(C)', y='HOEP', hue='Year',legend="full")
    plt.ylabel('HOEP ($)')
    plt.xlabel('Température (°C)')
    ax.set_ylim([0,94])
    ax.set_xlim([-35,35])
    ax.legend(title='Année')
    sns.despine()
    #plt.savefig('Relation_HOEP_Temp',dpi=1100, bbox_inches='tight')
    
with sns.axes_style("ticks", {"grid.color": "white"}):
    fig, ax = plt.subplots()
    sns.scatterplot(data=df,x='Temp(C)', y='MktDem', hue='Year',legend="full")
    plt.ylabel("Demande d'électricité MWh")
    plt.xlabel('Température (°C)')
    ax.legend(title='Année')
    ax.set_ylim([9000,26000])
    sns.despine()
    #plt.savefig('Relation_MktDem_Temp',dpi=1100, bbox_inches='tight')